# Delete

We are going to see how to delete one or multiple rows of a table based on some filtering criteria

## Imports

In [1]:
import psycopg2
import numpy as np
#import a folder in the parent directory
import sys
sys.path.append('../')
import dbtools.dbtools as qrs
import dbtools.delete as delete
from pathlib import Path
import tifffile as tiff

## Connection

In [2]:
try:
    # Connect to the PostgreSQL database
    conn = qrs.connect()
    print("Connected to the database")

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Connected to the database


## Loading the table from where we want to delete to check

In [3]:
table_name = 'materials'

data = qrs.get_data_metadata(table_name)

data

,id_material,name_material,layer_thickness_material,density_material,color_material,temperature_resistance_material
0,15,Fibra IM7 Resina M56,0.25 float,NaN,NaN,NaN
1,16,Material Hexcel 0.508 CPT,0.508 float,NaN,NaN,NaN
2,19,Prueba 6,0.1 float,1.5 g/cm3,blue string,250 celsius


## Filtering to decide what to delete

We want to delete all the measurements with the metadata key signal_type_measurement as amplitude.

In [4]:
filtered_data = data[data['name_material']== 'Prueba 6']

filtered_data

,id_material,name_material,layer_thickness_material,density_material,color_material,temperature_resistance_material
2,19,Prueba 6,0.1 float,1.5 g/cm3,blue string,250 celsius


## Deleting

In [5]:
delete.delete(conn, table_name, filtered_data['id_material'].values)

Deleted 1 rows from materials where id is in [19]
